In [1]:
import numpy as np
import pandas as pd
import time
from scipy import stats
from sklearn.preprocessing import MinMaxScaler
import joblib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
import matplotlib
import os
# %matplotlib notebook
%matplotlib inline

In [2]:
body_part = "head"
file_names = ['./data/training_data/head_data_1.log',
             './data/training_data/head_data_2.log',
             './data/training_data/head_data_3.log',
             './data/training_data/head_data_4.log',
             './data/training_data/head_data_5.log',
             './data/training_data/head_data_6.log']

# body_part = "shoulder_right"
# file_names = [
#              './data/training_data/shoulder_right_data_6.log',
#              './data/training_data/shoulder_right_data_7.log',
#              './data/training_data/shoulder_right_data_8.log',
#              './data/training_data/shoulder_right_data_9.log',
#              './data/training_data/shoulder_right_data_14.log',
#              './data/training_data/shoulder_right_data_15.log',
#              './data/training_data/shoulder_right_data_16.log',
#              './data/training_data/shoulder_right_data_17.log']

# body_part = "shoulder_left"
# file_names = ['./data/training_data/shoulder_left_data_1.log',
#               './data/training_data/shoulder_left_data_2.log',
#               './data/training_data/shoulder_left_data_3.log']

dataset = [pd.read_csv(f, delim_whitespace=True, header=0) for f in file_names]
dataset = [data[(np.abs(stats.zscore(data[["roll", "pitch", "yaw"]])) < 2.75).all(axis=1)] for data in dataset]
dataset_len = [len(data) for data in dataset]
dataset = pd.concat(dataset)

print('%d values'%(len(dataset)))

433845 values


In [3]:
dataset_len

[102093, 8746, 38589, 219455, 18399, 46563]

In [4]:
dataset = dataset.values[1:len(dataset)-1,0:]
# np.random.shuffle(dataset)

dataset = dataset[abs(dataset[:,12])!=0.0,:]
dataset = dataset[abs(dataset[:,13])!=0.0,:]
dataset = dataset[abs(dataset[:,14])!=0.0,:]
print('%d values after filtering outliers'%(len(dataset)))

euler_set = dataset[:, 12:15]
print('max euler ' + str(np.amax(euler_set)))
print('min euler ' + str(np.amin(euler_set)))
sensors_set = dataset[:, :12]

433843 values after filtering outliers
max euler 1.0645041825449684
min euler -0.9293592614230198


In [5]:
sin_cos_set = np.hstack([np.sin(euler_set), np.cos(euler_set)])

In [6]:
sensors_scaler = MinMaxScaler(feature_range=(-1., 1.))
sensors_set = sensors_scaler.fit_transform(sensors_set)

In [7]:
data_split = 0.8
split_idx = int(len(sensors_set)*data_split)

In [8]:
look_back = 10

data_in = []
data_out = []

start_idx = 0
for l in dataset_len:
    # Ignore the last batch
    for i in range(start_idx, start_idx+l-look_back*2):
        data_in.append(sensors_set[i:i+look_back])
        data_out.append(sin_cos_set[i+1:i+look_back+1])
    print(len(data_in))
    start_idx += l

102073
110799
149368
368803
387182
433725


In [9]:
data_split = 0.8
data_idx = np.arange(len(data_in))
np.random.shuffle(data_idx)
split_idx = int(len(data_in)*data_split)

train_in = np.array(data_in)[data_idx[:split_idx]]
train_out = np.array(data_out)[data_idx[:split_idx]]
test_in = np.array(data_in)[data_idx[split_idx:]]
test_out = np.array(data_out)[data_idx[split_idx:]]

In [10]:
train_in.shape

(346980, 10, 12)

## Train model

In [11]:
from nn_model import NeuralNetworkModel, LSTMNeuralNetworkModel

Instructions for updating:
non-resource variables are not supported in the long term


In [12]:
model_path = './output/'+body_part+'_lstm'

if not os.path.exists(model_path):
    os.makedirs(model_path)

model_name = model_path+'/best_model'
joblib.dump(sensors_scaler, model_path+'/scaler.pkl') 

model = LSTMNeuralNetworkModel(name=body_part, hidden_size=100, look_back=look_back)
# model = NeuralNetworkModel(name=body_part, hidden_size=100)
model.fit(x=train_in, y=train_out, x_val=test_in, y_val=test_out, save_path=model_name, 
         iteration=1000, patience=100, batch_size=1000)

/home/roboy/anaconda3/envs/roboy/lib/python3.8/site-packages/tensorflow/python/keras/legacy_tf_layers/core.py:171: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  warnings.warn('`tf.layers.dense` is deprecated and '
/home/roboy/anaconda3/envs/roboy/lib/python3.8/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1692: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '


#0000: Traing loss 0.34209, Valid loss 0.06837
#0010: Traing loss 0.02461, Valid loss 0.02424
#0020: Traing loss 0.02335, Valid loss 0.02317
#0030: Traing loss 0.01697, Valid loss 0.01687
#0040: Traing loss 0.00508, Valid loss 0.00498
#0050: Traing loss 0.00471, Valid loss 0.00477
#0060: Traing loss 0.00463, Valid loss 0.00475
#0070: Traing loss 0.00454, Valid loss 0.00454
#0080: Traing loss 0.00449, Valid loss 0.00442
#0090: Traing loss 0.00443, Valid loss 0.00437
#0100: Traing loss 0.00439, Valid loss 0.00434
#0110: Traing loss 0.00430, Valid loss 0.00443
#0120: Traing loss 0.00425, Valid loss 0.00424
#0130: Traing loss 0.00420, Valid loss 0.00417
#0140: Traing loss 0.00413, Valid loss 0.00416
#0150: Traing loss 0.00406, Valid loss 0.00412
#0160: Traing loss 0.00400, Valid loss 0.00401
#0170: Traing loss 0.00394, Valid loss 0.00397
#0180: Traing loss 0.00386, Valid loss 0.00391
#0190: Traing loss 0.00384, Valid loss 0.00391
#0200: Traing loss 0.00373, Valid loss 0.00379
#0210: Traing